### A3.3.1. Work Partitioning

$$
\text{Speedup} = \frac{1}{(1 - p) + \frac{p}{n}}
$$

**Amdahl's Law:** where $p$ is the parallelizable fraction and $n$ the number of processors.

**Explanation:**

**Work partitioning** divides a computation across threads or cores so they execute concurrently. The goal is to minimize total wall-clock time while avoiding synchronization overhead.

**Partitioning Strategies:**

| Strategy | Description | Use Case |
|----------|-------------|----------|
| Static (block) | Divide N items into N/P contiguous chunks | Uniform cost per item |
| Static (cyclic) | Round-robin assignment | Varying cost, no locality need |
| Dynamic | Workers pull from shared queue | Highly variable item cost |
| Work-stealing | Idle workers steal from busy workers' queues | Irregular task graphs |

**Key Considerations:**

- **Load balance** — even distribution of work across threads. Imbalance means some threads finish early and idle.
- **Granularity** — too-fine partitioning wastes time on synchronization; too-coarse limits parallelism.
- **Data locality** — partitioning should respect cache structure; threads should access nearby memory.
- **Amdahl's Law** — the serial fraction limits maximum speedup regardless of core count.

**Gustafson's Law (Scaled Speedup):**

$$\text{Speedup} = n - \alpha(n - 1)$$

where $\alpha$ is the serial fraction. As problem size grows with core count, parallel efficiency can remain high.

**Example:**

Summing a 1M-element array across 4 threads: split into 4 chunks of 250K, each thread computes a partial sum, then combine the 4 partial sums (a small serial reduction).

In [ ]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import time

SIZE = 10_000_000
data = np.random.rand(SIZE)


def partial_sum(chunk):
    return np.sum(chunk)


def parallel_sum(data, num_workers):
    chunks = np.array_split(data, num_workers)
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        partial_sums = list(executor.map(partial_sum, chunks))
    return sum(partial_sums)


serial_result = np.sum(data)

worker_counts = [1, 2, 4, 8]
print(f"Array size: {SIZE:,}")
print(f"Serial sum: {serial_result:.6f}\n")

for num_workers in worker_counts:
    result = parallel_sum(data, num_workers)
    chunk_size = SIZE // num_workers
    print(f"Workers: {num_workers}, chunk size: {chunk_size:,}, result: {result:.6f}")

print("\nAmdahl's Law predictions:")
parallel_fractions = [0.50, 0.75, 0.90, 0.95, 0.99]
core_counts = [2, 4, 8, 16, 64]

header = f"{'p':>6}" + "".join(f"{n:>8}" for n in core_counts)
print(header)
for parallel_fraction in parallel_fractions:
    serial_fraction = 1 - parallel_fraction
    speedups = [
        1 / (serial_fraction + parallel_fraction / cores)
        for cores in core_counts
    ]
    row = f"{parallel_fraction:>6.0%}" + "".join(f"{speedup:>8.2f}" for speedup in speedups)
    print(row)

**References:**

[📘 Hennessy, J. & Patterson, D. (2019). *Computer Architecture: A Quantitative Approach (6th ed.).* Morgan Kaufmann.](https://www.elsevier.com/books/computer-architecture/hennessy/978-0-12-811905-1)

[📘 McCool, M., Reinders, J. & Robison, A. (2012). *Structured Parallel Programming.* Morgan Kaufmann.](https://www.elsevier.com/books/structured-parallel-programming/mccool/978-0-12-415993-8)

---

[⬅️ Previous: Vectorization Reports](../02_Vectorization/02_vectorization_reports.ipynb) | [Next: Lock Contention ➡️](./02_lock_contention.ipynb)